In [ ]:
import os
import sys
import time
import uuid
import io
import base64
import tempfile
import shutil
import subprocess
import warnings
import IPython.display as ipd
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple, Union

# 警告を非表示にする
warnings.filterwarnings('ignore')

# 必要なライブラリをインストール
try:
    import ipywidgets as widgets
    import torch
    import torchaudio
    import numpy as np
    import sounddevice as sd
    import scipy.io.wavfile as wav
    from tqdm.auto import tqdm
    import soundfile as sf
    from IPython.display import display, Audio, HTML, Javascript
except ImportError:
    print("必要なライブラリをインストールしています...")
    !pip install -q ipywidgets torch torchaudio numpy tqdm soundfile huggingface_hub sounddevice scipy

    # Jupyter環境を初期化し直す必要がある場合の処理
    display(HTML("""
    <div style="background-color: #ffffcc; padding: 10px; border: 1px solid #ffcc00; border-radius: 5px;">
        <p><strong>注意:</strong> ライブラリがインストールされました。正常に動作させるには、このセルを実行した後、
        <b>Runtime > Restart runtime</b> を選択してノートブックを再起動してください。</p>
    </div>
    """))
    import ipywidgets as widgets
    import torch
    import torchaudio
    import numpy as np
    import sounddevice as sd
    import scipy.io.wavfile as wav
    from tqdm.auto import tqdm
    import soundfile as sf

# グローバル変数
MODEL_INITIALIZED = False
global_status = widgets.HTML(
    value='<div style="color: #777;">準備中...</div>'
)

# Fish Speech の初期化
print("Fish Speechモデルを初期化しています...")

# 必要なディレクトリ構造を作成
os.makedirs("checkpoints/fish-speech-1.5", exist_ok=True)
os.makedirs("references", exist_ok=True)
os.makedirs("outputs", exist_ok=True)

# モデルのダウンロード（存在しない場合のみ）
model_files = [
    "model.pth",
    "tokenizer.tiktoken", 
    "config.json",
    "firefly-gan-vq-fsq-8x1024-21hz-generator.pth"
]

def download_models():
    """モデルファイルをダウンロードする関数"""
    from huggingface_hub import hf_hub_download
    
    global_status.value = '<div style="color: #3498db;"><i class="fa fa-spinner fa-spin"></i> Fish Speech モデルをダウンロードしています...</div>'
    
    progress = widgets.IntProgress(
        value=0,
        min=0,
        max=len(model_files),
        description='ダウンロード:',
        bar_style='info',
        orientation='horizontal'
    )
    display(progress)
    
    for i, file in enumerate(model_files):
        if not os.path.exists(f"checkpoints/fish-speech-1.5/{file}"):
            try:
                global_status.value = f'<div style="color: #3498db;"><i class="fa fa-spinner fa-spin"></i> ファイルをダウンロード中: {file}...</div>'
                hf_hub_download(
                    repo_id="fishaudio/fish-speech-1.5",
                    filename=file,
                    local_dir="checkpoints/fish-speech-1.5",
                    local_dir_use_symlinks=False
                )
            except Exception as e:
                global_status.value = f'<div style="color: #e74c3c;">エラー: モデルのダウンロードに失敗しました: {e}</div>'
                raise e
        progress.value = i + 1
    
    global_status.value = '<div style="color: #2ecc71;"><i class="fa fa-check"></i> モデルのダウンロードが完了しました</div>'

try:
    download_models()
except Exception as e:
    global_status.value = f'<div style="color: #e74c3c;">エラー: {e}</div>'
    print(f"モデルのダウンロードに失敗しました: {e}")
    print("手動でモデルをダウンロードしてください:")
    print("https://huggingface.co/fishaudio/fish-speech-1.5")

# Fish Speech のモジュールをインポート
sys.path.append(os.getcwd())
try:
    from fish_speech.models.vqgan.inference import load_model as load_decoder_model
    from fish_speech.models.text2semantic.inference import launch_thread_safe_queue
    from fish_speech.inference_engine import TTSInferenceEngine
    from fish_speech.utils.schema import ServeTTSRequest, ServeReferenceAudio
except ImportError:
    global_status.value = '<div style="color: #e74c3c;">エラー: Fish Speech モジュールをロードできませんでした</div>'
    print("Fish Speech モジュールをロードできませんでした。")
    print("このノートブックが Fish Speech リポジトリのルートディレクトリで実行されていることを確認してください。")
    raise

# グローバル変数として推論エンジンを保持
inference_engine = None

def initialize_models():
    """TTS モデルを初期化する関数"""
    global inference_engine, MODEL_INITIALIZED
    
    if MODEL_INITIALIZED:
        return inference_engine
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    precision = torch.bfloat16
    compile_flag = False
    
    global_status.value = f'<div style="color: #3498db;"><i class="fa fa-spinner fa-spin"></i> モデルを初期化中 (デバイス: {device})...</div>'
    
    # LLama モデルの初期化
    global_status.value = '<div style="color: #3498db;"><i class="fa fa-spinner fa-spin"></i> LLama モデルを読み込んでいます...</div>'
    llama_queue = launch_thread_safe_queue(
        checkpoint_path="checkpoints/fish-speech-1.5",
        device=device,
        precision=precision,
        compile=compile_flag,
    )
    
    # Decoder モデルの初期化
    global_status.value = '<div style="color: #3498db;"><i class="fa fa-spinner fa-spin"></i> Decoder モデルを読み込んでいます...</div>'
    decoder_model = load_decoder_model(
        config_name="firefly_gan_vq",
        checkpoint_path="checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth",
        device=device,
    )
    
    # 推論エンジンの初期化
    global_status.value = '<div style="color: #3498db;"><i class="fa fa-spinner fa-spin"></i> 推論エンジンを初期化中...</div>'
    inference_engine = TTSInferenceEngine(
        llama_queue=llama_queue,
        decoder_model=decoder_model,
        precision=precision,
        compile=compile_flag,
    )
    
    # ウォームアップ実行
    global_status.value = '<div style="color: #3498db;"><i class="fa fa-spinner fa-spin"></i> モデルをウォームアップ中...</div>'
    list(inference_engine.inference(
        ServeTTSRequest(
            text="こんにちは",
            references=[],
            reference_id=None,
            max_new_tokens=1024,
            chunk_length=200,
            top_p=0.7,
            repetition_penalty=1.5,
            temperature=0.7,
            format="wav",
        )
    ))
    
    MODEL_INITIALIZED = True
    global_status.value = '<div style="color: #2ecc71;"><i class="fa fa-check"></i> モデルの初期化が完了しました！</div>'
    return inference_engine

# 音声クローン関数
def create_voice_clone(audio_file, text, clone_id=None):
    """音声ファイルとテキストから音声クローンを作成する関数"""
    if clone_id is None:
        clone_id = str(uuid.uuid4().hex)
    
    # リファレンスディレクトリの作成
    ref_dir = os.path.join("references", clone_id)
    os.makedirs(ref_dir, exist_ok=True)
    
    # 音声ファイルをコピー
    audio_ext = os.path.splitext(audio_file)[1]
    ref_audio_path = os.path.join(ref_dir, f"reference{audio_ext}")
    shutil.copy(audio_file, ref_audio_path)
    
    # テキストファイルを作成
    ref_text_path = os.path.join(ref_dir, "reference.lab")
    with open(ref_text_path, "w", encoding="utf-8") as f:
        f.write(text)
    
    return clone_id

# 利用可能な話者リストを取得する関数
def get_available_speakers():
    """references フォルダにある話者IDのリストを取得する"""
    speakers = []
    ref_dir = Path("references")
    if ref_dir.exists():
        speakers = [d.name for d in ref_dir.iterdir() if d.is_dir()]
    return speakers

# 音声生成関数
def generate_speech(text, reference_id=None, reference_audio=None, reference_text=None, 
                   max_new_tokens=1024, chunk_length=200, top_p=0.7, 
                   repetition_penalty=1.2, temperature=0.7, seed=None, 
                   use_memory_cache="on", status_callback=None):
    """テキストから音声を生成する関数"""
    global inference_engine
    
    # モデルが初期化されていない場合は初期化
    if inference_engine is None:
        inference_engine = initialize_models()
    
    references = []
    if reference_audio and reference_text:
        with open(reference_audio, "rb") as audio_file:
            audio_bytes = audio_file.read()
        references = [ServeReferenceAudio(audio=audio_bytes, text=reference_text)]
    
    req = ServeTTSRequest(
        text=text,
        reference_id=reference_id if reference_id else None,
        references=references,
        max_new_tokens=max_new_tokens,
        chunk_length=chunk_length,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        temperature=temperature,
        seed=seed if seed and seed > 0 else None,
        use_memory_cache=use_memory_cache,
        format="wav",
    )
    
    # 音声生成
    audio_data = None
    error_msg = None
    
    if status_callback:
        status_callback('<div style="color: #3498db;"><i class="fa fa-spinner fa-spin"></i> 音声を生成中...</div>')
    
    for result in inference_engine.inference(req):
        if result.code == "final":
            audio_data = result.audio
            if status_callback:
                status_callback('<div style="color: #2ecc71;"><i class="fa fa-check"></i> 音声生成が完了しました！</div>')
            break
        elif result.code == "error":
            error_msg = str(result.error)
            if status_callback:
                status_callback(f'<div style="color: #e74c3c;"><i class="fa fa-times"></i> エラー: {error_msg}</div>')
            break
    
    if error_msg:
        return None, error_msg
    
    if audio_data is None:
        if status_callback:
            status_callback('<div style="color: #e74c3c;"><i class="fa fa-times"></i> エラー: 音声が生成されませんでした</div>')
        return None, "音声が生成されませんでした"
    
    # 音声データを返す
    sample_rate, audio = audio_data
    return (sample_rate, audio), None

# 音声ファイルを保存する関数
def save_audio(audio_data, filename="generated_audio.wav"):
    """音声データをファイルに保存する関数"""
    sample_rate, audio = audio_data
    output_path = os.path.join("outputs", filename)
    sf.write(output_path, audio, sample_rate)
    return output_path

# ダウンロードボタン機能のために、一時ファイルを作成して HTML でダウンロードリンクを生成
def get_download_link(audio_data, filename="generated_audio.wav"):
    """音声データからダウンロードリンクを生成する関数"""
    sample_rate, audio = audio_data
    
    # WAVデータを一時ファイルとして保存
    temp_dir = tempfile.mkdtemp()
    temp_file = os.path.join(temp_dir, filename)
    sf.write(temp_file, audio, sample_rate)
    
    # ファイルを読み込んでBase64エンコード
    with open(temp_file, "rb") as f:
        audio_data_b64 = base64.b64encode(f.read()).decode()
    
    # 一時ディレクトリを削除
    shutil.rmtree(temp_dir)
    
    # Base64エンコードされたデータを使ってダウンロードリンクを生成
    download_link = f"""
    <a href="data:audio/wav;base64,{audio_data_b64}" 
       download="{filename}" 
       class="download-button"
       style="display: inline-block; 
              background-color: #4CAF50; 
              color: white; 
              padding: 8px 16px; 
              text-align: center; 
              text-decoration: none; 
              font-size: 14px; 
              margin: 4px 2px; 
              cursor: pointer; 
              border-radius: 4px;">
        <i class="fa fa-download"></i> ダウンロード: {filename}
    </a>
    """
    return download_link

# UI 構築
def build_ui():
    """ipywidgetsを使用してUIを構築する関数"""
    # スタイルの設定
    style = {'description_width': '150px'}
    layout = widgets.Layout(width='100%')
    
    # FontAwesomeのCDNを読み込み
    display(HTML("""
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">
    """))
    
    # ヘッダー
    header = widgets.HTML(
        value="""
        <h1 style="color: #2c3e50; font-family: Arial, sans-serif;">
            <i class="fas fa-microphone-alt"></i> Fish Speech TTS アプリケーション
        </h1>
        <p style="color: #7f8c8d; font-family: Arial, sans-serif; margin-bottom: 20px;">
          FishAudioが開発したテキスト音声変換アプリケーション
        </p>
        <hr style="margin-bottom: 20px;">
        """
    )
    
    # 1. テキスト入力タブ
    text_input = widgets.Textarea(
        value='こんにちは、私はAIアシスタントです。お手伝いできることがあれば教えてください。',
        placeholder='ここにテキストを入力してください',
        description='テキスト:',
        style=style,
        layout=layout,
        rows=5
    )
    
    speaker_dropdown = widgets.Dropdown(
        options=[('なし', None)] + [(s, s) for s in get_available_speakers()],
        value=None,
        description='話者ID:',
        style=style,
        layout=widgets.Layout(width='50%')
    )
    
    generate_button = widgets.Button(
        description='音声生成',
        button_style='primary',
        icon='play',
        layout=widgets.Layout(width='auto')
    )
    
    refresh_speakers_button = widgets.Button(
        description='話者リスト更新',
        button_style='info',
        icon='sync',
        layout=widgets.Layout(width='auto')
    )
    
    output_audio = widgets.Output()
    download_area = widgets.HTML()
    status_area = widgets.HTML()
    
    # 2. テンプレート音声クローンタブ
    template_text = widgets.HTML(
        value="""
        <div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 15px;">
            <p style="margin: 0;">次のテンプレートテキストを録音してください:</p>
            <blockquote style="margin: 10px 0; padding: 10px; border-left: 3px solid #007bff; background-color: #e9ecef;">
                魚は水の中で泳ぎ、鳥は空を飛びます。私たちは言葉を通じてコミュニケーションをとります。
                この音声は、私の声をクローンするためのサンプルです。
            </blockquote>
        </div>
        """
    )
    
    # 録音用コンポーネント
    duration_slider = widgets.IntSlider(
        value=5, 
        min=3, 
        max=30, 
        step=1, 
        description='録音時間(秒):',
        style=style,
        layout=widgets.Layout(width='50%')
    )
    
    record_button = widgets.Button(
        description='録音開始',
        button_style='danger',
        icon='microphone',
        layout=widgets.Layout(width='auto')
    )
    
    stop_button = widgets.Button(
        description='録音停止',
        button_style='warning',
        icon='stop',
        layout=widgets.Layout(width='auto', display='none')
    )
    
    recording_status = widgets.HTML()
    recording_audio = widgets.Output()
    
    clone_id_input = widgets.Text(
        value='',
        placeholder='任意のID (空白の場合は自動生成)',
        description='クローンID:',
        style=style,
        layout=widgets.Layout(width='50%')
    )
    
    create_clone_button = widgets.Button(
        description='音声クローン作成',
        button_style='primary',
        icon='clone',
        layout=widgets.Layout(width='auto')
    )
    
    clone_output = widgets.HTML()
    
    # 3. 音声アップロードクローンタブ
    upload_audio = widgets.FileUpload(
        accept='.wav,.mp3,.ogg,.flac',
        description='音声ファイル:',
        style=style,
        layout=layout,
        multiple=False
    )
    
    reference_text = widgets.Textarea(
        value='',
        placeholder='アップロードした音声に対応するテキストを入力してください',
        description='参照テキスト:',
        style=style,
        layout=layout,
        rows=3
    )
    
    upload_clone_id = widgets.Text(
        value='',
        placeholder='任意のID (空白の場合は自動生成)',
        description='クローンID:',
        style=style,
        layout=widgets.Layout(width='50%')
    )
    
    upload_clone_button = widgets.Button(
        description='アップロードして音声クローン作成',
        button_style='primary',
        icon='upload',
        layout=widgets.Layout(width='auto', margin='10px 0px')
    )
    
    upload_status = widgets.HTML()
    upload_clone_output = widgets.HTML()
    
    # 4. 詳細設定タブ
    advanced_settings = widgets.VBox([
        widgets.IntSlider(
            value=200,
            min=0,
            max=300,
            step=10,
            description='チャンク長:',
            style=style,
            layout=layout
        ),
        widgets.IntSlider(
            value=1024,
            min=256,
            max=2048,
            step=128,
            description='最大トークン数:',
            style=style,
            layout=layout
        ),
        widgets.FloatSlider(
            value=0.7,
            min=0.1,
            max=1.0,
            step=0.05,
            description='Top-p:',
            style=style,
            layout=layout
        ),
        widgets.FloatSlider(
            value=1.2,
            min=1.0,
            max=2.0,
            step=0.1,
            description='繰り返しペナルティ:',
            style=style,
            layout=layout
        ),
        widgets.FloatSlider(
            value=0.7,
            min=0.1,
            max=1.0,
            step=0.05,
            description='温度:',
            style=style,
            layout=layout
        ),
        widgets.IntText(
            value=0,
            description='シード:',
            style=style,
            layout=widgets.Layout(width='50%')
        ),
        widgets.Dropdown(
            options=[('オン', 'on'), ('オフ', 'off')],
            value='on',
            description='メモリキャッシュ:',
            style=style,
            layout=widgets.Layout(width='50%')
        ),
    ])
    
    # ステータスエリア
    status_box = widgets.VBox([
        widgets.Label('システムステータス:'),
        global_status
    ], layout=widgets.Layout(margin='10px 0'))
    
    # タブの作成
    tabs = widgets.Tab()
    tabs.children = [
        widgets.VBox([
            text_input,
            widgets.HBox([speaker_dropdown, refresh_speakers_button]),
            generate_button,
            status_area,
            output_audio,
            download_area
        ]),
        widgets.VBox([
            template_text,
            widgets.HBox([duration_slider]),
            widgets.HBox([record_button, stop_button]),
            recording_status,
            recording_audio,
            clone_id_input,
            create_clone_button,
            clone_output
        ]),
        widgets.VBox([
            upload_audio,
            reference_text,
            upload_clone_id,
            upload_clone_button,
            upload_status,
            upload_clone_output
        ]),
        advanced_settings
    ]
    
    tabs.set_title(0, '🎤 テキスト音声変換')
    tabs.set_title(1, '🔊 テンプレート音声クローン')
    tabs.set_title(2, '📁 音声アップロードクローン')
    tabs.set_title(3, '⚙️ 詳細設定')
    
    # 録音用の変数
    recording_data = {"audio": None, "thread": None, "running": False}
    
    # イベントハンドラの設定
    def on_generate_click(b):
        status_area.value = ""
        download_area.value = ""
        
        # ボタンを無効化
        b.disabled = True
        b.icon = 'spinner'
        b.description = '生成中...'
        
        with output_audio:
            output_audio.clear_output()
            print("音声を生成しています...")
            
            # 詳細設定から値を取得
            chunk_length = advanced_settings.children[0].value
            max_new_tokens = advanced_settings.children[1].value
            top_p = advanced_settings.children[2].value
            repetition_penalty = advanced_settings.children[3].value
            temperature = advanced_settings.children[4].value
            seed = advanced_settings.children[5].value
            use_memory_cache = advanced_settings.children[6].value
            
            # 音声生成
            audio_data, error = generate_speech(
                text_input.value,
                reference_id=speaker_dropdown.value,
                max_new_tokens=max_new_tokens,
                chunk_length=chunk_length,
                top_p=top_p,
                repetition_penalty=repetition_penalty,
                temperature=temperature,
                seed=seed,
                use_memory_cache=use_memory_cache,
                status_callback=lambda msg: setattr(status_area, 'value', msg)
            )
            
            output_audio.clear_output()
            
            if error:
                status_area.value = f'<div style="color: #e74c3c;"><i class="fa fa-times"></i> エラー: {error}</div>'
                # ボタンを再有効化
                b.disabled = False
                b.icon = 'play'
                b.description = '音声生成'
                return
            
            # 音声を再生
            display(Audio(data=audio_data[1], rate=audio_data[0]))
            
            # ダウンロードリンクを表示
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            filename = f"generated_{timestamp}.wav"
            save_audio(audio_data, filename)
            download_area.value = get_download_link(audio_data, filename)
            
            # ボタンを再有効化
            b.disabled = False
            b.icon = 'play'
            b.description = '音声生成'
    
    def start_recording(b):
        recording_status.value = '<div style="color: #e74c3c;"><i class="fa fa-circle"></i> 録音中...</div>'
        record_button.layout.display = 'none'
        stop_button.layout.display = 'block'
        
        def record_thread_function():
            try:
                fs = 44100  # サンプルレート（Hz）
                seconds = duration_slider.value
                
                recording_data["running"] = True
                # 録音開始（モノラル録音）
                recording = sd.rec(int(seconds * fs), samplerate=fs, channels=1, dtype='float32')
                
                # 録音終了まで待機
                for i in range(seconds):
                    if not recording_data["running"]:
                        sd.stop()
                        break
                    time.sleep(1)
                    recording_status.value = f'<div style="color: #e74c3c;"><i class="fa fa-circle"></i> 録音中... {i+1}/{seconds}秒</div>'
                
                if recording_data["running"]:
                    sd.wait()  # 録音終了まで待機
                    recording_data["running"] = False
                
                # wavファイルとして保存
                filename = "recording.wav"
                if len(recording) > 0:
                    recording_data["audio"] = (fs, recording)
                    wav.write(filename, fs, (recording * 32767).astype(np.int16))
                    recording_status.value = f'<div style="color: #2ecc71;"><i class="fa fa-check"></i> 録音完了: {filename}</div>'
                    
                    # 録音した音声を表示
                    with recording_audio:
                        recording_audio.clear_output()
                        display(Audio(data=recording, rate=fs))
                else:
                    recording_status.value = '<div style="color: #e74c3c;"><i class="fa fa-times"></i> 録音エラー: 録音データがありません</div>'
            except Exception as e:
                recording_status.value = f'<div style="color: #e74c3c;"><i class="fa fa-times"></i> 録音エラー: {str(e)}</div>'
            finally:
                record_button.layout.display = 'block'
                stop_button.layout.display = 'none'
        
        # 録音スレッドを開始
        recording_data["thread"] = threading.Thread(target=record_thread_function)
        recording_data["thread"].start()
    
    def stop_recording(b):
        recording_data["running"] = False
        recording_status.value = '<div style="color: #f39c12;"><i class="fa fa-stop"></i> 録音を停止しています...</div>'
    
    def on_create_clone_click(b):
        clone_output.value = ""
        
        # ボタンを無効化
        b.disabled = True
        b.icon = 'spinner'
        b.description = 'クローン作成中...'
        
        if not recording_data["audio"]:
            clone_output.value = '<div style="color: #e74c3c;"><i class="fa fa-times"></i> エラー: 録音データがありません。テンプレートを録音してください。</div>'
            b.disabled = False
            b.icon = 'clone'
            b.description = '音声クローン作成'
            return
        
        # 録音データを一時ファイルに保存
        temp_dir = tempfile.mkdtemp()
        temp_audio_file = os.path.join(temp_dir, "template.wav")
        
        fs, audio_data = recording_data["audio"]
        wav.write(temp_audio_file, fs, (audio_data * 32767).astype(np.int16))
        
        # テンプレートテキスト
        template_value = "魚は水の中で泳ぎ、鳥は空を飛びます。私たちは言葉を通じてコミュニケーションをとります。この音声は、私の声をクローンするためのサンプルです。"
        
        # クローンID
        clone_id = clone_id_input.value if clone_id_input.value else None
        
        # 音声クローン作成
        try:
            clone_output.value = '<div style="color: #3498db;"><i class="fa fa-spinner fa-spin"></i> 音声クローンを作成中...</div>'
            clone_id = create_voice_clone(temp_audio_file, template_value, clone_id)
            clone_output.value = f'''
            <div style="background-color: #d4edda; color: #155724; padding: 10px; border-radius: 4px; margin-top: 10px;">
                <i class="fa fa-check-circle"></i> 音声クローンが作成されました！<br>
                クローンID: <b>{clone_id}</b><br>
                このIDを「テキスト音声変換」タブの話者IDドロップダウンで選択することで、あなたの声でテキストを読み上げることができます。
            </div>
            '''
            
            # 話者リストを更新
            speaker_dropdown.options = [('なし', None)] + [(s, s) for s in get_available_speakers()]
        except Exception as e:
            clone_output.value = f'<div style="color: #e74c3c;"><i class="fa fa-times"></i> エラー: {str(e)}</div>'
        finally:
            # 一時ファイルを削除
            shutil.rmtree(temp_dir)
            # ボタンを再有効化
            b.disabled = False
            b.icon = 'clone'
            b.description = '音声クローン作成'
    
    def on_upload_clone_click(b):
        upload_clone_output.value = ""
        upload_status.value = ""
        
        # ボタンを無効化
        b.disabled = True
        b.icon = 'spinner'
        b.description = '処理中...'
        
        if not upload_audio.value:
            upload_status.value = '<div style="color: #e74c3c;"><i class="fa fa-times"></i> エラー: 音声ファイルをアップロードしてください。</div>'
            b.disabled = False
            b.icon = 'upload'
            b.description = 'アップロードして音声クローン作成'
            return
        
        if not reference_text.value:
            upload_status.value = '<div style="color: #e74c3c;"><i class="fa fa-times"></i> エラー: 参照テキストを入力してください。</div>'
            b.disabled = False
            b.icon = 'upload'
            b.description = 'アップロードして音声クローン作成'
            return
        
        # ファイル名を取得
        file_info = next(iter(upload_audio.value.values()))
        file_name = file_info['metadata']['name']
        
        # 一時ファイルに保存
        temp_dir = tempfile.mkdtemp()
        temp_audio_file = os.path.join(temp_dir, file_name)
        
        with open(temp_audio_file, "wb") as f:
            f.write(file_info['content'])
        
        # クローンID
        clone_id = upload_clone_id.value if upload_clone_id.value else None
        
        # 音声クローン作成
        try:
            upload_status.value = '<div style="color: #3498db;"><i class="fa fa-spinner fa-spin"></i> 音声クローンを作成中...</div>'
            clone_id = create_voice_clone(temp_audio_file, reference_text.value, clone_id)
            upload_clone_output.value = f'''
            <div style="background-color: #d4edda; color: #155724; padding: 10px; border-radius: 4px; margin-top: 10px;">
                <i class="fa fa-check-circle"></i> 音声クローンが作成されました！<br>
                クローンID: <b>{clone_id}</b><br>
                このIDを「テキスト音声変換」タブの話者IDドロップダウンで選択することで、あなたの声でテキストを読み上げることができます。
            </div>
            '''
            
            # 話者リストを更新
            speaker_dropdown.options = [('なし', None)] + [(s, s) for s in get_available_speakers()]
        except Exception as e:
            upload_clone_output.value = f'<div style="color: #e74c3c;"><i class="fa fa-times"></i> エラー: {str(e)}</div>'
        finally:
            # 一時ファイルを削除
            shutil.rmtree(temp_dir)
            # ボタンを再有効化
            b.disabled = False
            b.icon = 'upload'
            b.description = 'アップロードして音声クローン作成'
    
    def on_refresh_speakers(b):
        # ボタンを無効化
        b.disabled = True
        b.icon = 'spinner'
        
        # 話者リストを更新
        speaker_dropdown.options = [('なし', None)] + [(s, s) for s in get_available_speakers()]
        
        # ボタンを再有効化
        b.disabled = False
        b.icon = 'sync'
    
    # イベントハンドラの登録
    generate_button.on_click(on_generate_click)
    record_button.on_click(start_recording)
    stop_button.on_click(stop_recording)
    create_clone_button.on_click(on_create_clone_click)
    upload_clone_button.on_click(on_upload_clone_click)
    refresh_speakers_button.on_click(on_refresh_speakers)
    
    # UIを表示
    return widgets.VBox([header, status_box, tabs])

# モデルの初期化
try:
    inference_engine = initialize_models()
except Exception as e:
    global_status.value = f'<div style="color: #e74c3c;"><i class="fa fa-exclamation-triangle"></i> モデルの初期化に失敗しました: {e}</div>'

# メインアプリケーションの構築と表示
app = build_ui()
display(app)